这里是回答bpe的相关问题的地方

# Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)
(a) Train a byte-level BPE tokenizer on the TinyStories dataset, using a maximum vocabulary size
of 10,000. Make sure to add the TinyStories <|endoftext|> special token to the vocabulary.
Serialize the resulting vocabulary and merges to disk for further inspection. How many hours
and memory did training take? What is the longest token in the vocabulary? Does it make sense?
Resource requirements: ≤30 minutes (no GPUs), ≤ 30GB RAM

Hint You should be able to get under 2 minutes for BPE training using multiprocessing during
pretokenization and the following two facts:

### (a) The <|endoftext|> token delimits documents in the data files.

### (b) The <|endoftext|> token is handled as a special case before the BPE merges are applied.

(b) Profile your code. What part of the tokenizer training process takes the most time?

训练bpe的时候使用multiprocess进行并行化加速了，使用了两种写法，一种是mp.Pool, 但是这个程序卡死了；另一个方法是创建了一个list，里面放上每一个进程，但是会出现程序运行一会就挂起的情况，，使用top -H -p processs_numer 发现在sleeping， cpu利用率是零

但是我使用倒排索引成功的将原本二三十分钟的训练bpe的过程压缩到了6分钟左右。原理是：我们每一次选出出现次数最多的byte pair之后，需要遍历整个文本的子序列进行更新（将相应的byte合并在一起），但是有了索引我们就能精准的修改那些拥有这个byte pair对的子序列。

时间瓶颈： 对文本进行pre-token和merge的更新

# Problem (train_bpe_expts_owt): BPE Training on OpenWebText (2 points)
(a) Train a byte-level BPE tokenizer on the OpenWebText dataset, using a maximum vocabulary
size of 32,000. Serialize the resulting vocabulary and merges to disk for further inspection. What
is the longest token in the vocabulary? Does it make sense?
Resource requirements: ≤12 hours (no GPUs), ≤ 100GB RAM
Deliverable: A one-to-two sentence response.
(b) Compare and contrast the tokenizer that you get training on TinyStories versus OpenWebText.
Deliverable: A one-to-two sentence response

首先是训练这个我的本机5点多一点的，根本不行。训练速度很慢，于是我在autodl上面租了一台强大的cpu跑成功了：
训练过程有以下几点：
-  刚开始训练的时候十分慢，因为刚开始有的pair非常非常大，首先找到pair之后对于相应地子序列地合并操作会很多
- 随着后面整个pair_count(记录目前每一个pair出现地次数)在不断地减小, 因为一个pair合并导致相应subword2count，inverted_index地修改就会越来越少，于是就会越来越快 

# Problem (tokenizer_experiments): Experiments with tokenizers (4 points）
(a) Sample 10 documents from TinyStories and OpenWebText. Using your previously-trained TinyS-
tories and OpenWebText tokenizers (10K and 32K vocabulary size, respectively), encode these
sampled documents into integer IDs. What is each tokenizer’s compression ratio (bytes/token)?

这个在tokenzier那一小节可以更好地展示